<a href="https://colab.research.google.com/github/JesusRamirezGamarra/Python-DeCodificador/blob/main/Afterclass_Bases_de_Datos_y_Sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conexión a la base de datos

In [ ]:
import sqlite3

conexion = sqlite3.connect('base.db')
cursor = conexion.cursor()


cursor.execute("PRAGMA foreign_keys = ON")  # Activación de claves foráneas para Sqlite
conexion.commit()  # Guarda cambios

# Creación de tablas

### Pais

In [ ]:
cursor.execute("""
    CREATE TABLE Pais (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL UNIQUE
    )
""")

print("Tabla creada")

### Producto

In [ ]:
cursor.execute("""
    CREATE TABLE Producto (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL UNIQUE
    )
""")

print("Tabla creada")

### Cliente

In [ ]:
cursor.execute("""
    CREATE TABLE Cliente (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL,
        apellido TEXT NOT NULL,
        nacimiento TEXT,
        pais_origen_id REFERENCES Pais(id) ON DELETE SET NULL
        )
""")

print("Tabla creada")

### Cliente_Producto

In [ ]:
cursor.execute("""
    CREATE TABLE Cliente_Producto (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        cliente_id REFERENCES Cliente(id) ON DELETE CASCADE,
        producto_id REFERENCES Producto(id) ON DELETE CASCADE
        )
""")

print("Tabla creada")

# Creación de registros

### Paises

In [ ]:
cursor.execute("INSERT INTO Pais (nombre) VALUES (?)", ("Argentina",))
conexion.commit()

print("Paises creados")

In [ ]:
# Dará un error por la restricción UNIQUE cuando se creó la tabla País.

cursor.execute("INSERT INTO Pais (nombre) VALUES (?)", ("Argentina",))
conexion.commit()

print("Paises creados")

In [ ]:
paises = ["Brasil", "Chile", "México", "Ecuador", "Colombia", "Uruguay"]

for pais in paises:
    cursor.execute("INSERT INTO Pais (nombre) VALUES (?)", (pais,))
conexion.commit()

print("Paises creados")

In [ ]:
cursor.execute("SELECT * FROM Pais")
print(cursor.fetchall())

In [ ]:
cursor.execute("SELECT * FROM Pais")
print(cursor.fetchone())

In [ ]:
cursor.execute("SELECT nombre, id FROM Pais")
print(cursor.fetchmany(3))

### Productos

In [ ]:
productos = [("azúcar",), ("leche",), ("queso",)]

sql = "INSERT INTO Producto (nombre) VALUES (?)"
cursor.executemany(sql, productos)
print("Productos creados")

In [ ]:
cursor.execute("SELECT * FROM Producto")
items = cursor.fetchall()
for id, producto in items:
    print(id, producto)

### Cliente

In [ ]:
print("CREANDO CLIENTE")

nombre = input("Nombre: ")
if not nombre:
    nombre = None
apellido = input("Apellido: ")
if not apellido:
    apellido = None
nacimiento = input("Fecha de nacimiento (aaaa-mm-dd): ")

# SELECT
print("País de origen:")
cursor.execute("SELECT * FROM Pais")
items = cursor.fetchall()
for id, pais in items:
    print(f"\t{id}: {pais}")

while True:
    entrada = int(input("Seleccione opción: "))
    if entrada in [x[0] for x in items]:
        break
    else:
        continue
pais_origen_id = entrada

# INSERT
sql = "INSERT INTO Cliente (nombre, apellido, nacimiento, pais_origen_id) VALUES (?,?,?,?)"
cursor.execute(sql, (nombre, apellido, nacimiento, pais_origen_id))
conexion.commit()

print("Cliente creado")

In [ ]:
cursor.execute("SELECT * FROM Cliente")
print(cursor.fetchall())

In [ ]:
cursor.execute("""
            SELECT Cliente.id, Cliente.nombre, apellido, nacimiento, Pais.nombre
            FROM Cliente INNER JOIN Pais ON Cliente.pais_origen_id = Pais.id
        """)
items = cursor.fetchall()

encabezado = f"{'ID':<5} | {'NOMBRE':<10} | {'APELLIDO':<15} | {'NACIMIENTO':<15} | {'PAIS ORIGEN':<15}"
print(encabezado + "\n" + len(encabezado) * "=")
for item in items:
    print(f"{item[0]:<5} | {item[1]:<10} | {item[2]:<15} | {item[3]:<15} | {item[4]:<15}")

### Cliente_Producto

In [ ]:
print("CLIENTE COMPRA PRODUCTO")

# SELECT
print("Clientes:")
cursor.execute("SELECT * FROM Cliente")
items = cursor.fetchall()
for item in items:
    print(f"\t{item[0]}: {item[1]} {item[2]}")

while True:
    entrada = int(input("Seleccione opción: "))
    if entrada in [x[0] for x in items]:
        break
    else:
        continue
if entrada == "":
    entrada = None
cliente_id = entrada

print("Productos:")
cursor.execute("SELECT * FROM Producto")
items = cursor.fetchall()
for id, producto in items:
    print(f"\t{id}: {producto}")

while True:
    entrada = int(input("Seleccione opción: "))
    if entrada in [x[0] for x in items]:
        break
    else:
        continue
if entrada == "":
    entrada = None
producto_id = entrada

# INSERT
sql = "INSERT INTO Cliente_Producto (cliente_id, producto_id) VALUES (?,?)"
cursor.execute(sql, (cliente_id, producto_id))
conexion.commit()
print("Cliente - Transacción exitosa")

In [ ]:
cursor.execute("SELECT * FROM Cliente_Producto")
print(cursor.fetchall())

In [ ]:
cursor.execute("""
            SELECT Cliente_Producto.id, Cliente.nombre, Cliente.apellido, Producto.nombre
            FROM Cliente_Producto 
            INNER JOIN Cliente ON Cliente_Producto.cliente_id = Cliente.id
            INNER JOIN Producto ON Cliente_Producto.producto_id = Producto.id
        """)
items = cursor.fetchall()

encabezado = f"{'ID':<5} | {'CLIENTE':<25} | {'PRODUCTO':<15}"
print(encabezado + "\n" + len(encabezado) * "=")
for item in items:
    cliente = item[1] + " " + item[2]
    print(f"{item[0]:<5} | {cliente:<25} | {item[3]:15}")

# Consultas con condiciones

In [ ]:
cursor.execute("""
            SELECT Cliente_Producto.id, Cliente.nombre, Cliente.apellido, Producto.nombre
            FROM Cliente_Producto 
            INNER JOIN Cliente ON Cliente_Producto.cliente_id = Cliente.id
            INNER JOIN Producto ON Cliente_Producto.producto_id = Producto.id
            ORDER BY Producto.nombre DESC
        """)
items = cursor.fetchall()
for item in items:
    print(item)

In [ ]:
cursor.execute("""
            SELECT Cliente_Producto.id, Cliente.nombre, Cliente.apellido, Producto.nombre
            FROM Cliente_Producto 
            INNER JOIN Cliente ON Cliente_Producto.cliente_id = Cliente.id
            INNER JOIN Producto ON Cliente_Producto.producto_id = Producto.id
            WHERE Producto.nombre = "leche"
        """)
items = cursor.fetchall()
for item in items:
    print(item)

In [ ]:
cursor.execute("""
            SELECT Cliente_Producto.id, Cliente.nombre, Cliente.apellido, Producto.nombre
            FROM Cliente_Producto 
            INNER JOIN Cliente ON Cliente_Producto.cliente_id = Cliente.id
            INNER JOIN Producto ON Cliente_Producto.producto_id = Producto.id
            WHERE Cliente.nombre LIKE "%cint%"
        """)
items = cursor.fetchall()
for item in items:
    print(item)

# Modificación de registros

In [ ]:
print("Modificando cliente.")
apellido_cambiar = input("Ingrese apellido a cambiar: ")
apellido_nuevo = input("Ingrese apellido nuevo: ")

sql = "UPDATE Cliente SET apellido = ? WHERE apellido = ?"
cursor.execute(sql, (apellido_nuevo, apellido_cambiar))
conexion.commit()
print("Registro modificado")

In [ ]:
cursor.execute("SELECT * FROM Cliente")
items = cursor.fetchall()
for item in items:
    print(item)

# Eliminación de registros

In [ ]:
print("Eliminando producto.")
producto_eliminar = input("Ingrese producto a eliminar: ")

sql = "DELETE FROM Producto WHERE nombre = ?"
cursor.execute(sql, (producto_eliminar,))
conexion.commit()
print("Registro eliminado")

In [ ]:
cursor.execute("SELECT * FROM Producto")
items = cursor.fetchall()
for item in items:
    print(item)

In [ ]:
sql = "DELETE FROM Pais WHERE nombre = ?"
cursor.execute(sql, ("Argentina",))
conexion.commit()
print("Registro eliminado")

In [ ]:
cursor.execute("SELECT * FROM Pais")
items = cursor.fetchall()
for item in items:
    print(item)

In [ ]:
cursor.execute("SELECT * FROM Cliente")
items = cursor.fetchall()
for item in items:
    print(item)

In [ ]:
cursor.execute("SELECT * FROM Cliente_Producto")
items = cursor.fetchall()
for item in items:
    print(item)

In [ ]:
cursor.execute("""
            SELECT Cliente_Producto.id, Cliente.nombre, Cliente.apellido, Producto.nombre
            FROM Cliente_Producto 
            INNER JOIN Cliente ON Cliente_Producto.cliente_id = Cliente.id
            INNER JOIN Producto ON Cliente_Producto.producto_id = Producto.id
        """)
items = cursor.fetchall()

encabezado = f"{'ID':<5} | {'CLIENTE':<25} | {'PRODUCTO':<15}"
print(encabezado + "\n" + len(encabezado) * "=")
for item in items:
    cliente = item[1] + " " + item[2]
    print(f"{item[0]:<5} | {cliente:<25} | {item[3]:15}")